In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

from mpl_toolkits.mplot3d import Axes3D

import warnings
warnings.filterwarnings("ignore")

In [2]:
df3 = pd.read_excel('./C题数据.xlsx', sheet_name='data3')

In [3]:
# data = torch.tensor(df3.iloc[:, : 5].values, dtype=torch.float)
# target1 = torch.tensor(df3.iloc[:, 5].values, dtype=torch.float).reshape(-1, 1)
# target2 = torch.tensor(df3.iloc[:, 6].values, dtype=torch.float).reshape(-1, 1)

In [4]:
data = torch.tensor(df3.iloc[:, : 5].values, dtype=torch.float)
target = torch.tensor(df3.iloc[:, 5:7].values, dtype=torch.float)

In [5]:
x_train,x_val,y_train,y_val = train_test_split(data,target,
                                               test_size=0.2,random_state=20)
train = torch.cat([x_train, y_train], dim=1)
val = torch.cat([x_val, y_val], dim=1)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.net1 = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
        
        self.net2 = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )
        
    def forward(self, x):
        y = self.net1(x)
        # z = self.net2(torch.cat([x, y.reshape(-1, 1)], dim=1))
        z = self.net2(x)
        return y, z

In [7]:
# model = Net()
model = torch.load('./model2.pkl')

In [8]:
loss_func = torch.nn.MSELoss()

In [9]:
opt = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
for i in range(1):
    model.train()
    for d in train:
        y, z = model(d[:5])
        loss1 = loss_func(y, d[5])
        loss2 = loss_func(z, d[6])

        # 优化器模型
        opt.zero_grad()
        loss1.backward()
        loss2.backward()
        opt.step()

    model.eval()
    total_test_loss1 = 0
    total_test_loss2 = 0
    with torch.no_grad():
        for da in val:
            y, z = model(da[:5])
            loss1 = loss_func(y, da[5])
            loss2 = loss_func(z, da[6])
            total_test_loss1 += loss1.item()
            total_test_loss2 += loss2.item()
        if i%100==0:
            print("{}验证集上的loss1: {}, loss2:{}".format(i/100,total_test_loss1, total_test_loss2))
            if total_test_loss1+total_test_loss2 <= 140:
                torch.save(model, './model.pkl')
print("训练完成!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

0.0验证集上的loss1: 88.1139189065434, loss2:75.52541904710233
训练完成!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [11]:
b = model(data)

In [12]:
b = torch.cat(b, dim=1)

In [13]:
torch.save(model, './model1.pkl')

In [14]:
x = torch.tensor([[38, 850, 2.6099, 96.0257, 87.0805],
[33, 950, 2.5807, 95.9860, 87.1232],
[28, 1150, 2.7612, 96.2081, 86.8333],
[23, 1250, 2.7318, 96.1754, 86.8843],
[38, 1250, 1.4372, 91.0524, 85.0118],
[33, 1150, 3.0018, 96.4407, 86.3757],
[28, 950, 2.3442, 95.5899, 87.3854],
[23, 850, 1.9144, 94.2767, 87.1976]])

In [15]:
ans = torch.cat(model(x), dim=1)
ans

tensor([[28.8716, 49.6199],
        [29.1648, 48.5316],
        [27.8573, 49.2182],
        [28.1552, 66.3750],
        [24.2829, 48.0835],
        [25.9168, 45.7054],
        [30.9950, 49.8815],
        [33.9843, 52.6471]], grad_fn=<CatBackward0>)

In [16]:
ans[:, 1].max()

tensor(66.3750, grad_fn=<MaxBackward1>)